# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import glob
import joblib
import warnings
from tensorflow.keras.models import load_model

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
ROOT = '' 
SOURCE_PATH = f'Source'
RESULT_PATH = f'Result'



RNN_PATH = f"{RESULT_PATH}/RNN"

# Function to download stock data from yfinance

In [ ]:
def Download_Stock(Code, start, end , dest = None):

    data = yf.download( Code , start = start, end = end, interval="1d")
    data = data.reset_index()


    data.columns = ['Date', 'Close', 'High', 'Low', "Open", "Volumn"]
    data['Code'] = Code


    df_output = data.copy()

    for col in data.columns:

        if pd.api.types.is_datetime64_any_dtype(data[col]):

            df_output[col] = data[col].dt.date


    if not dest == None:

        df_output.to_csv(dest, index = False)


    return df_output

# Function to plot model training result

In [ ]:
def Plot_Result(data, title, Code, model = "", name = "", IsSave = False, IsDisplay = True):

    save_path = f'{RESULT_PATH}/{model}/{Code}/{name}'


    x = data['Date']
    y = data['Close']


    plt.figure(figsize=(15, 8))

    plt.plot(x, y, color='blue', label='Actual y', alpha=0.5 , linewidth = 1)

    try:

        y_pred = data['Close_Pred']

        plt.plot(x, y_pred, color='red', label='Predicted y', alpha=0.8, linewidth = 1 )

    except:

        pass



    plt.ylabel(f"Stock Price of {Code}")
    plt.xlabel('Date')
    plt.title(title)

    plt.legend()
    plt.grid(True)

    if IsSave:

        plt.savefig(save_path , dpi=300)


    if IsDisplay:

        plt.show()

    else:

        plt.close()


# Function to generate future consecutive business dates

In [ ]:
def Generate_Predict_Date(input_df, num_days):

    start_date = pd.to_datetime(input_df.iloc[-1]['Date'])

    dates = pd.date_range(start=start_date, periods=num_days + 1, freq='B')[1:]


    output_df = pd.concat([input_df, pd.DataFrame({'Date': dates.date}) ], ignore_index=True)


    return output_df

# Function to get RNN models

In [ ]:
def Get_RNN_Mode(Code):

    CurrPath = f"{RNN_PATH}/{Code}/Current"
    rnn_models = glob.glob(f'{CurrPath}/*.keras',  recursive=True)
    scaler = joblib.load(f'{CurrPath}/RNN_{Code}_Scaler.pkl')


    models = []

    for path in rnn_models:

        model = load_model(path)

        models.append(model)

    return models,scaler